# Data Exploration and Modification

In [32]:
%matplotlib inline 
import os, sys
import json
import networkx as nx
import random
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import numpy as np
from networkx.readwrite import json_graph
from sklearn.preprocessing import MinMaxScaler

In [33]:
import pandas as pd
dfs = pd.read_excel('2020-03-18_Krogan_SARSCoV2_27baits.xlsx')  
dfs.head()

,Bait,Preys,PreyGene,MIST,BFDR,AvgSpec,FoldChange,Abundance
0,SARS-CoV2 E,O00203,AP3B1,0.963550,0.00,4.67,46.67,0.000489
1,SARS-CoV2 E,O60885,BRD4,0.978488,0.00,2.67,26.67,0.000252
2,SARS-CoV2 E,P25440,BRD2,0.906593,0.00,7.00,70.00,0.001006
3,SARS-CoV2 E,Q6UX04,CWC27,0.893109,0.03,2.67,26.67,0.000637
4,SARS-CoV2 E,Q86VM9,ZC3H18,0.796415,0.00,2.67,26.67,0.000366


In [34]:
def BaitNameEdit(x):
    return x.split()[1]

dfs.Bait = dfs.Bait.apply(BaitNameEdit)
dfs.head()

,Bait,Preys,PreyGene,MIST,BFDR,AvgSpec,FoldChange,Abundance
0,E,O00203,AP3B1,0.963550,0.00,4.67,46.67,0.000489
1,E,O60885,BRD4,0.978488,0.00,2.67,26.67,0.000252
2,E,P25440,BRD2,0.906593,0.00,7.00,70.00,0.001006
3,E,Q6UX04,CWC27,0.893109,0.03,2.67,26.67,0.000637
4,E,Q86VM9,ZC3H18,0.796415,0.00,2.67,26.67,0.000366


In [35]:
scaler = MinMaxScaler()
dfs['MIST'] = scaler.fit_transform(dfs['MIST'].values.reshape(-1,1))
dfs['FoldChange'] = scaler.fit_transform(dfs['FoldChange'].values.reshape(-1,1))
dfs.head()

,Bait,Preys,PreyGene,MIST,BFDR,AvgSpec,FoldChange,Abundance
0,E,O00203,AP3B1,0.922048,0.00,4.67,0.017824,0.000489
1,E,O60885,BRD4,0.960230,0.00,2.67,0.006684,0.000252
2,E,P25440,BRD2,0.776467,0.00,7.00,0.030819,0.001006
3,E,Q6UX04,CWC27,0.742003,0.03,2.67,0.006684,0.000637
4,E,Q86VM9,ZC3H18,0.494856,0.00,2.67,0.006684,0.000366


In [18]:
baitList = dfs.Bait.to_list()
preyList = dfs.Preys.to_list()
nodeList = list(set(baitList+preyList))

In [19]:
graph = nx.DiGraph()
graph.add_nodes_from(nodeList)

In [20]:
dfs = dfs.reset_index() 
for index, row in dfs.iterrows():
    prey = row.Preys
    bait = row.Bait
    mist = row.MIST
    graph.add_edge(prey, bait, weight = mist)

In [21]:
foldDict = {i:-1 for i in nodeList}
for index, row in dfs.iterrows():
    foldDict[row.Preys] = row.FoldChange

In [22]:
nx.set_node_attributes(graph, foldDict, "FoldChange") # adding degree
nx.set_node_attributes(graph, dict(graph.in_degree(weight="weight")), "in_strength") # adding indegree
nx.set_node_attributes(graph, dict(graph.out_degree(weight="weight")), "out_strength") # adding outdegree
nx.set_node_attributes(graph, nx.in_degree_centrality(graph), "indegree_centrality") # adding indegree_centrality
nx.set_node_attributes(graph, nx.out_degree_centrality(graph), "outdegree_centrality") # adding outdegree_centrality
# nx.set_node_attributes(graph, nx.clustering(graph), "node_clustering_coefficient") # adding clustering coefficient
nx.set_node_attributes(graph, nx.closeness_centrality(graph), "closeness_centrality") # adding closeness_centrality

In [23]:
from networkx.readwrite import json_graph
import json
data1 = json_graph.node_link_data(graph)
s1 = json.dumps(data1)

In [24]:
with open('json_data.json', 'w') as outfile:
    outfile.write(s1)